<a href="https://colab.research.google.com/github/maxim920/Machine-Learning/blob/main/Kosenko_lab7_1_4_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from surprise import Dataset
from surprise import Reader
import pandas as pd

data = Dataset.load_builtin('ml-100k')


raw_data = data.raw_ratings

# Виведемо перші 5 рядків сирих даних
for i in range(5):
    print(raw_data[i])




('196', '242', 3.0, '881250949')
('186', '302', 3.0, '891717742')
('22', '377', 1.0, '878887116')
('244', '51', 2.0, '880606923')
('166', '346', 1.0, '886397596')


In [6]:
from surprise import KNNBasic, SVD
from surprise.model_selection import cross_validate

# Алгоритм 1: KNNBasic
knn = KNNBasic()
knn_results = cross_validate(knn, data, measures=['MAE'], cv=5, verbose=True)

# Алгоритм 2: SVD
svd = SVD()
svd_results = cross_validate(svd, data, measures=['MAE'], cv=5, verbose=True)


Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
MAE (testset)     0.7718  0.7740  0.7754  0.7735  0.7683  0.7726  0.0024  
Fit time          0.64    0.53    0.37    0.35    0.53    0.48    0.11    
Test time         4.05    4.07    3.20    3.19    4.10    3.72    0.43    
Evaluating MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
MAE (testset)     0.7383  0.7398  0.7377  0.7385  0.7315  0.7372  0.0029  
Fit time          1.43    1.40    1.47    1.42    1.85    1.52    0.17    
Test time

In [7]:
from surprise.model_selection import GridSearchCV

# Підбір параметрів для KNN
param_grid_knn = {'k': [10, 20, 30], 'sim_options': {'name': ['msd', 'cosine'], 'user_based': [False]}}
gs_knn = GridSearchCV(KNNBasic, param_grid_knn, measures=['mae'], cv=5)
gs_knn.fit(data)

# Підбір параметрів для SVD
param_grid_svd = {'n_factors': [50, 100], 'lr_all': [0.002, 0.005], 'reg_all': [0.02, 0.05]}
gs_svd = GridSearchCV(SVD, param_grid_svd, measures=['mae'], cv=5)
gs_svd.fit(data)


Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix.

In [8]:
# Виведення найкращих результатів
print(f"KNN Best MAE: {gs_knn.best_score['mae']}")
print(f"SVD Best MAE: {gs_svd.best_score['mae']}")

if gs_knn.best_score['mae'] < gs_svd.best_score['mae']:
    best_algo = gs_knn.best_estimator['mae']
else:
    best_algo = gs_svd.best_estimator['mae']

print(f"Обраний найкращий алгоритм: {best_algo}")


KNN Best MAE: 0.7711033853995326
SVD Best MAE: 0.736629622477073
Обраний найкращий алгоритм: <surprise.prediction_algorithms.matrix_factorization.SVD object at 0x7f3957d7fbb0>


In [10]:
# Виведення рекомендації для конкретного користувача
trainset = data.build_full_trainset()
best_algo.fit(trainset)

user_id = str(345)
items = trainset.all_items()
items_rated_by_user = [j for (j, _) in trainset.ur[trainset.to_inner_uid(user_id)]]

recommendations = []
for item_id in items:
    if item_id not in items_rated_by_user:
        est = best_algo.predict(user_id, trainset.to_raw_iid(item_id)).est
        recommendations.append((item_id, est))

# Виведення топ-10 рекомендованих фільмів
recommendations.sort(key=lambda x: x[1], reverse=True)
top_10_movies = recommendations[:10]
top_10_movies


[(538, 4.844126952190348),
 (180, 4.727212996077026),
 (240, 4.70931010042799),
 (612, 4.663074214914015),
 (174, 4.5944829604008355),
 (189, 4.592946409778483),
 (185, 4.592300883525253),
 (541, 4.590934539908932),
 (367, 4.566793824293146),
 (260, 4.542457479007795)]

In [11]:

import os

movie_file_path = os.path.expanduser('~/.surprise_data/ml-100k/ml-100k/u.item')
movie_names = {}

with open(movie_file_path, encoding='ISO-8859-1') as f:
    for line in f:
        fields = line.split('|')
        movie_id = int(fields[0])
        movie_name = fields[1]
        movie_names[movie_id] = movie_name



In [13]:
# Відображення назв замість їх ID
top_10_movies_with_names = [(movie_names[item_id], rating) for item_id, rating in top_10_movies]


for movie_name, rating in top_10_movies_with_names:
    print(f"Фільм: {movie_name}, Передбачена оцінка: {rating:.2f}")


Фільм: Anastasia (1997), Передбачена оцінка: 4.84
Фільм: Apocalypse Now (1979), Передбачена оцінка: 4.73
Фільм: Beavis and Butt-head Do America (1996), Передбачена оцінка: 4.71
Фільм: Lost Horizon (1937), Передбачена оцінка: 4.66
Фільм: Raiders of the Lost Ark (1981), Передбачена оцінка: 4.59
Фільм: Grand Day Out, A (1992), Передбачена оцінка: 4.59
Фільм: Psycho (1960), Передбачена оцінка: 4.59
Фільм: Mortal Kombat (1995), Передбачена оцінка: 4.59
Фільм: Clueless (1995), Передбачена оцінка: 4.57
Фільм: Event Horizon (1997), Передбачена оцінка: 4.54


Висновки

У ході порівняння алгоритмів для побудови рекомендаційної системи на основі набору даних MovieLens 100k було використано алгоритми KNN та SVD. За результатами крос-валідації з 5 фолдів, алгоритм SVD продемонстрував кращу точність: його середня абсолютна похибка (MAE) склала 0.7372, тоді як у KNN цей показник був вищим — 0.7726. Алгоритм SVD не тільки точніше передбачає оцінки, але й має стабільніші результати (менша стандартна відхилення). Попри те, що час навчання SVD трохи довший (середній час — 1.52 секунди проти 0.48 у KNN), SVD є кращим вибором завдяки вищій точності. Тому він був обраний для фінальної системи рекомендацій.